### Objective
Create San Diego Block group boundaries along with opportunity and competition score.

In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection, GeoAccessor

import sys
sys.path.append('../')
sys.path.append('../../')
from utils import get_config, format_json
from feature_layer_utils import get_enrichment_variables, create_feature_layer

import pandas as pd

from arcgis.geometry  import Geometry

import json

from itertools import chain

In [2]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

##### Reading the score file created for block groups and opportuity/competition score.

In [4]:
score_df = pd.read_csv("../resources/block_grp_fips_opportunity_competition_score.csv", dtype=str)
print(f"Shape: {score_df.shape}")
score_df.head(3)

Shape: (2057, 2)


,fips,Opportunity_Competition_Score_Scaled
0,060730100101,0.7058869122579962
1,060730100102,0.4500799730642858
2,060730100103,0.6938168920778944


##### Reading in san diego county block group boundaries feature layer for those geometries

In [5]:
sd_block_group_lyr = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a38898/FeatureServer/0")

In [6]:
sd_block_group_df = sd_block_group_lyr.query(as_df=True, return_geometry=True)
sd_block_group_df.drop(columns=['FID','Shape__Area','Shape__Length','blockgroup','county_fip','state_abbr','state_fips','tract_fips']
                      , inplace=True)
print(f"Shape: {sd_block_group_df.shape}")
sd_block_group_df.head(3)


Shape: (2057, 2)


,SHAPE,fips
0,"{""rings"": [[[-13045413.2286, 3863333.31369999]...",060730001001
1,"{""rings"": [[[-13045214.9685, 3863075.4548], [-...",060730001002
2,"{""rings"": [[[-13043127.839, 3863072.5428], [-1...",060730002011


In [7]:
merged_df = sd_block_group_df.merge(score_df, on='fips',how='left')
merged_df['Opportunity_Competition_Score_Scaled'] =  merged_df['Opportunity_Competition_Score_Scaled'].fillna('0')
merged_df.rename(columns={'Opportunity_Competition_Score_Scaled':'oc_score'}, inplace=True)
merged_df['oc_score'] = merged_df['oc_score'].astype(float)
merged_df

,SHAPE,fips,oc_score
0,"{'rings': [[[-13045413.2286, 3863333.31369999]...",060730001001,0.398806
1,"{'rings': [[[-13045214.9685, 3863075.4548], [-...",060730001002,0.332207
2,"{'rings': [[[-13043127.839, 3863072.5428], [-1...",060730002011,0.289465
3,"{'rings': [[[-13043560.4266, 3862026.86880002]...",060730002021,0.384101
4,"{'rings': [[[-13044064.8153, 3862217.59810002]...",060730002022,0.411878
...,...,...,...
2052,"{'rings': [[[-13054989.7152, 3913804.0585], [-...",060730221012,0.267620
2053,"{'rings': [[[-13055545.9784, 3908498.1588], [-...",060730221021,0.427506
2054,"{'rings': [[[-13054612.6758, 3910254.47430001]...",060730221022,0.319779
2055,"{'rings': [[[-13056063.9482, 3910470.4437], [-...",060730221023,0.427729


In [8]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2057 entries, 0 to 2056
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SHAPE     2057 non-null   object 
 1   fips      2057 non-null   object 
 2   oc_score  2057 non-null   float64
dtypes: float64(1), object(2)
memory usage: 64.3+ KB


Publish feature layer

In [9]:
%%time
my_new_featurelayer = merged_df.spatial.to_featurelayer(title='San Diego Block Groups Food Business opportunity and competition Score', 
                                                      gis=gis, 
                                                      folder='nourish_gis',
                                                      tags=['SanDiego', 'Potential Score','Opportunity'],
                                                     )
my_new_featurelayer

CPU times: total: 1.25 s
Wall time: 15.4 s


<Item title:"San Diego Block Groups Food Business opportunity and competition Score" type:Feature Layer Collection owner:akale_UCSD>